## Pipeline: 
- NOTEVENT["HADM_ID"]
- DIAGNOSES_ICD["HADM_ID"] 
- DIAGNOSES_ICD["ICD9_CODE"] (First 5 ["SEQ_NUM"]) 
- D_ICD_DIAGNOSES["ICD9_CODE"]
- D_ICD_DIAGNOSES["LONG_TITLE"]
- Bio.Entrez query diagnoses (long title)
- pubmed id list 
- metapub find + download pdfs
- pypdfium2 read pdfs

In [13]:
from Bio.Entrez import esearch
from Bio import Entrez
import pandas as pd
import os
from urllib.request import urlretrieve
import metapub
import pypdfium2 as pdfium

In [ ]:
icd9_list = [41401, 2851, 25000, 4019, 2720]

In [14]:
Entrez.email = 'minhvu031722@gmail.com'
#term = "Tuberculous pneumonia [any form], tubercle bacilli not found by bacteriological or histological examination, but tuberculosis confirmed by other methods [inoculation of animals]"
#handle = Entrez.esearch(db="pubmed", term=term, retmax=200, sort='relevance')
#records = Entrez.read(handle)
#print(records['IdList'])

In [12]:
import os
files = os.listdir("pdf")
files

['0309',
 'record-141',
 'record-37',
 '0005',
 '0037',
 'record-25',
 '0365',
 '0418',
 '0177',
 '0009',
 'record-106',
 '0349',
 'record-14',
 'record-124',
 '0217',
 'record-74',
 '0129',
 '0397',
 '0113',
 'record-84',
 '0361',
 '0201',
 '0289',
 '0241',
 '0093',
 '0285',
 '0460',
 '0237',
 '0401',
 '0193',
 'record-26',
 'record-13',
 '0317',
 '0225',
 '0442',
 '0393',
 'record-27',
 'record-52',
 'record-53',
 'record-32',
 '0325',
 '0301',
 '0337',
 '0117',
 'record-28',
 '0221',
 '0001',
 '0233',
 'record-108',
 '0149',
 'record-24',
 'record-142',
 '0061',
 'record-51',
 'record-82',
 '0313',
 '0329',
 'record-29',
 'record-19',
 '0065',
 '0021',
 'record-55',
 'record-48',
 'record-122',
 'record-18',
 '0245',
 '0209',
 'record-16',
 '0293',
 '0097',
 'record-68',
 'record-175',
 'record-123',
 '0427',
 '0253',
 'record-177',
 '0445',
 '0229',
 'record-73',
 '0161',
 'record-105',
 '0273',
 'record-17',
 '0457',
 '0125',
 '0389',
 '0436',
 'record-30',
 '0381',
 '0045',
 '010

In [53]:
icd9toicd10 = {}
with open("icd9to10dictionary.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        parts = line.strip().split("|")
        icd9 = parts[0].replace(".", "")
        desc = parts[-1]
        icd9toicd10[icd9] = desc
icd9toicd10

{'0010': 'Cholera due to Vibrio cholerae 01 biovar cholerae',
 '0011': 'Cholera due to Vibrio cholerae 01 biovar eltor',
 '0019': 'Cholera unspecified',
 '0020': 'Typhoid fever unspecified',
 '0021': 'Paratyphoid fever A',
 '0022': 'Paratyphoid fever B',
 '0023': 'Paratyphoid fever C',
 '0029': 'Paratyphoid fever unspecified',
 '0030': 'Salmonella enteritis',
 '0031': 'Salmonella sepsis',
 '00320': 'Localized salmonella infection unspecified',
 '00321': 'Salmonella meningitis',
 '00322': 'Salmonella pneumonia',
 '00323': 'Salmonella arthritis',
 '00324': 'Salmonella osteomyelitis',
 '00329': 'Salmonella with other localized infection',
 '0038': 'Other specified salmonella infections',
 '0039': 'Salmonella infection unspecified',
 '0040': 'Shigellosis due to Shigella dysenteriae',
 '0041': 'Shigellosis due to Shigella flexneri',
 '0042': 'Shigellosis due to Shigella boydii',
 '0043': 'Shigellosis due to Shigella sonnei',
 '0048': 'Other shigellosis',
 '0049': 'Shigellosis unspecified',


In [85]:
process_term(icd9toicd10["0010"], 1)

'Cholera due to Vibrio cholerae 01 biovar'

In [69]:
def process_term(term, n):
    tokens = term.split()
    return " ".join(tokens[:-n])

In [82]:
for f in files:
    if f not in redo_files:
        continue
    print(f)
    path = f"pdf/{f}"
    with open(path + "/diagnosis.txt", "r") as f:
        diags = f.readlines()
    n = min(len(diags), 3)
    for diag in diags[:n]:
        parts = diag.split(":")
        icd9 = parts[0].strip()
        term = parts[1].strip()
        if icd9 not in redo_icd9 or icd9 not in icd9toicd10:
            continue
        term = process_term(term, 5)
        pdf_path = path + "/" + icd9
        if not os.path.isdir(pdf_path):
            os.mkdir(pdf_path)
        handle = Entrez.esearch(db="pubmed", term=term, retmax=200, sort='relevance')
        records = Entrez.read(handle)
        pmid_list = records['IdList']
        get_pdf(pdf_path, pmid_list, 3)

record-37
0241
0117
record-51
0313
record-55
record-122
0209
0097
record-21


In [118]:
redo_files = []
redo_icd9 = []
for f in files:
    dirs = os.listdir("pdf/" + f)
    dirs = [d for d in dirs if ".txt" not in d and "ipynb" not in d]
    for d in dirs:
        path = "pdf/" + f + "/" + d
        fn = len([d for d in os.listdir(path) if "ipynb" not in d])
        if fn < 3:
            print(path, fn)
            redo_files.append(f)
            redo_icd9.append(d)
redo_files = list(set(redo_files))
redo_icd9 = [f for f in redo_icd9 if "ipynb" not in f]

In [119]:
len(redo_icd9)

0

In [115]:
term = "Closed fracture of shaft of radius"
handle = Entrez.esearch(db="pubmed", term=term, retmax=200, sort='relevance')
records = Entrez.read(handle)
print(len(records['IdList']))
pdf_path = "pdf/record-21/81321"
get_pdf(pdf_path, records['IdList'], 3)

115


In [98]:
#another_path = '/content/Articles/'
#another_path = './pdf'
#pmid_list = records['IdList']

def get_pdf(path, pmid_list, max_file):
    count = 0
    files_list = os.listdir(path)
    files_list = [f.replace(".pdf", "") for f in files_list]
    count += len(files_list)
    for query in pmid_list:
        if query in files_list:
            count += 1
            if count >= max_file:
                break
            else:
                continue
        url = None
        try:
            url = metapub.FindIt(query).url
        except Exception as e:
            continue
        if url:
            try:
                out_file = os.path.join(path, query)
                urlretrieve(url, out_file + '.pdf') 
                count += 1
                if count >= max_file:
                    break
            except Exception as ex:
                continue

In [1]:
from metapub import PubMedFetcher
fetch = PubMedFetcher()

pmids = ['33845964', '25857560', '28149579', '35613080', '29629687']
articles = {}
for pmid in pmids:
    articles[pmid] = fetch.article_by_pmid(pmid)

2023-12-28 23:14:42 moe metapub.config[3116461] WARNING NCBI_API_KEY was not set.


In [11]:
articles['33845964'].content

<Element PubmedArticle at 0x7fb7c53fe100>

In [21]:
pdf = pdfium.PdfDocument('pdf/29789497.pdf')

In [23]:
print(pdf[0].get_textpage().get_text_range())

cancers
Review
Treating p53 Mutant Aggregation-Associated Cancer
Mathumai Kanapathipillai
Department of Mechanical Engineering, University of Michigan-Dearborn, Dearborn, MI 48128, USA;
mathumai@umich.edu
Received: 7 April 2018; Accepted: 21 May 2018; Published: 23 May 2018


Abstract: p53 is a tumor suppressor protein. Under stressful conditions, p53 tightly regulates cell
growth by promoting apoptosis and DNA repair. When p53 becomes mutated, it loses its function,
resulting in abnormal cell proliferation and tumor progression. Depending on the p53 mutation,
it has been shown to form aggregates leading to negative gain of function of the protein. p53 mutant
associated aggregation has been observed in several cancer tissues and has been shown to promote
tumor growth. Recent studies show correlation between p53 mutant aggregation, functional loss,
and tumor growth. Moreover, p53 aggregation has been observed in biopsies, patient tissues,
and in vivo studies. Given the fact th

# Test Triggers Llama 2

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

from transformers import pipeline
import torch

device = torch.device("cuda")

text_gen = pipeline(
    task="text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=1000,
    device=device,
    do_sample=True,
    temperature=0.5,
    top_p=0.5,
)

2023-11-10 03:30:04 moe torch.distributed.nn.jit.instantiator[2520461] INFO Created a temporary directory at /scratch/35571/tmpjpholioa
2023-11-10 03:30:04 moe torch.distributed.nn.jit.instantiator[2520461] INFO Writing /scratch/35571/tmpjpholioa/_remote_module_non_scriptable.py


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
#prompt = f"<s>[INST] <<SYS>>\n{}\n<</SYS>>\n\n[/INST]</s>"
emr = "He was given Unasyn for empiric antibiotic treatment . At the time , blood , urine and bile was sent for culture , which were all negative . A chest x-ray was also negative for any pneumonia . Postoperatively , Mr. Cordano also suffered from low urine output ranges between 25 to 30 cc per hour . Fractional excretion of sodium was less than 1% suggesting a prerenal etiology . The sodium creatinine peaked at 2.0 and the patient was treated with boluses of intravenous fluid and intravenous albumin infusion was also administered . He responded well to this regimen and made adequate urine output. He was given Zosyn for empiric treatment . Gram stain and culture of the ascites failed to identify any organism . He was weaned from oxygen requirement and was continued on normal saline boluses and. intravenous albumin for treatment of prerenal azotemia . Somatostatin and midodrine were also given for this condition and he also received packed red blood cells . On postoperative day number ten , the patient returned to the floor with improved renal function and without oxygen requirement . The somatostatin and the midodrine were discontinued and the regimen of Lasix and spironolactone was started for diuresis."
triggers = "low urine output ranges between 25 to 30 cc per hour, prerenal azotemia"
abstract = articles['33845964'].abstract

In [6]:
prompt = [
"""<<SYS>>
You're a doctor and you were given the following EMRs by another doctor. You need to find some clinical triggers in these EMRs that you think important and need more infomation. 
<</SYS>>

Please find the clinical triggers for the following EMRs.
EMR: "{auxiliary}"
Clinical triggers found:
"""]
#print(prompt[0].format(emr=emr, triggers=triggers, auxiliary=abstract))
inst = prompt[0].format(auxiliary=abstract)

In [61]:
output = text_gen(inst)
print(output[0]['generated_text'])

<<SYS>>
You're a doctor and you were given the following EMRs by another doctor. You need to find some clinical triggers in these EMRs that you think important and need more infomation. 
<</SYS>>

Please find the clinical triggers for the following EMRs.
EMR: "Thyroid tuberculosis is a rare disease. Its incidence is low even in countries where prevalence of pulmonary tuberculosis is high (0.1-0.4%). In literature, there are only a few cases which were diagnosed as thyroid tuberculosis. It can be explained by a high resistance of the thyroid gland to infectious processes. However, the prevalence of tuberculosis has increased worldwide and thyroid involvement can be a primary manifestation of the disease. The incidence of extrapulmonary tuberculosis has been showing a progressive increase in the recent years(Barnes and Weatherstone, 1979). The most frequent clinical presentation is a solitary thyroid nodule that may present as a cystic nodule. It may also present as thyroid abscess with 

In [7]:
print(inst)

<<SYS>>
You're a doctor and you were given the following EMRs by another doctor. You need to find some clinical triggers in these EMRs that you think important and need more infomation. 
<</SYS>>

Please find the clinical triggers for the following EMRs.
EMR: "Thyroid tuberculosis is a rare disease. Its incidence is low even in countries where prevalence of pulmonary tuberculosis is high (0.1-0.4%). In literature, there are only a few cases which were diagnosed as thyroid tuberculosis. It can be explained by a high resistance of the thyroid gland to infectious processes. However, the prevalence of tuberculosis has increased worldwide and thyroid involvement can be a primary manifestation of the disease. The incidence of extrapulmonary tuberculosis has been showing a progressive increase in the recent years(Barnes and Weatherstone, 1979). The most frequent clinical presentation is a solitary thyroid nodule that may present as a cystic nodule. It may also present as thyroid abscess with 